In [5]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import re
import string

In [2]:
# Charger le fichier CSV en utilisant la fonction définie
Data = pd.read_csv('./dataset/training.1600000.processed.noemoticon.csv', encoding='latin1', header=None)
Data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
# Récupérer la première colonne dans une variable target
target = Data.iloc[:, 0]
# Récupérer la dernière colonne dans une variable text
text = Data.iloc[:, -1]

# Concaténer les deux variables
New_Data = pd.concat([target, text], axis=1)

# Enregistrer le résultat dans un fichier CSV
New_Data.to_csv('New_Data.csv', index=False, header=False)


In [ ]:
# Télécharger les données nécessaires pour NLTK
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Prétraitement

In [4]:
# Fonction pour nettoyer un tweet
def clean_tweet(tweet):

    # Convertir en minuscules
    tweet = tweet.lower()

    # Supprimer les balises HTML
    tweet = re.sub(r'<.*?>', '', tweet)

    # Supprimer les URL
    tweet = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', tweet)

    # Supprimer les mentions
    tweet = re.sub(r'@[^\s]+', '', tweet)

    # Supprimer les hashtags et leur mot suivant
    tweet = re.sub(r'#\w+\b\s*', '', tweet)

    # Normalisation todaaaay ==> today
    tweet = re.sub(r'([A-Za-z])\1{2,}', r'\1', tweet)

    # Supprimer les caractères non alphabétiques et la ponctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    stop_words.update(['new', 'way'])  # Ajouter des mots personnalisés à supprimer
    tweet = ' '.join(word for word in tweet.split() if word not in stop_words)

    # Racinisation des mots
    stemmer = PorterStemmer()
    tweet = ' '.join(stemmer.stem(word) for word in tweet.split())

    # Supprimer les caractères speciaux
    tweet = ''.join(char for char in tweet if char in string.printable)

    return tweet

In [5]:
# text_clean = text.apply(clean_tweet)
# text_clean = text.apply(clean_tweet)
# print(text)
# print(text_clean)
#Exemple d'utilisation
tweet = "todaaaaaaaaay @gmail that aw that bummer shoulda got david carr"
cleaned_tweet = clean_tweet(tweet)
print("Tweet original :", tweet)
print("Tweet nettoyé :", cleaned_tweet)

Tweet original : todaaaaaaaaay @gmail that aw that bummer shoulda got david carr
Tweet nettoyé : today aw bummer shoulda got david carr


In [6]:
dataframe = pd.read_csv('New_Data.csv', encoding='latin1', header=None)
dataframe.head()

,0,1
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [7]:
from nltk.tokenize import TweetTokenizer
import numpy as np
# Initialiser le tokenizer
tknzr = TweetTokenizer()

text_clean = dataframe[1].apply(clean_tweet)
dataframe = pd.DataFrame(np.column_stack([dataframe, text_clean]), columns=['target', 'text_tweet', 'text_clean'])

# Enregistrer le DataFrame dans un fichier CSV
dataframe.to_csv('cleaned_data.csv', index=False)

# Appliquer la tokenisation sur la deuxième colonne (index 1)
tokens_column = text_clean.apply(tknzr.tokenize)

# Utiliser np.column_stack() pour empiler les colonnes contenant les tokens
dataframe = pd.DataFrame(np.column_stack([dataframe, tokens_column]), columns=['target', 'text_tweet', 'text_clean', 'tokens'])

# Afficher les premières lignes pour vérification
dataframe.head()

# Enregistrer le DataFrame dans un fichier CSV
dataframe.to_csv('tokenize_data.csv', index=False, header=False)

# Construction Du Vocabulaire

In [8]:
#Construction de vocabulaire
from collections import Counter
def create_vocab():
    # Compter le nombre d'occurrences de chaque mot dans l'ensemble des tweets
    word_counts = Counter()

    for tokens in dataframe['tokens']:
        word_counts.update(tokens)

    # Filtrer les mots qui apparaissent au moins K fois dans le corpus de tweets
    K = 2000  # Choisir empiriquement K
    vocab = [word for word, count in word_counts.items() if count >= K]

    # Enregistrer les mots sélectionnés dans un fichier vocab.txt
    with open('vocab.txt', 'w', encoding='utf-8') as f:
        for word in vocab:
            f.write(word + '\n')

    # Mapper chaque mot prétraité à son index dans la liste de vocabulaire
    word_to_index = {word: idx for idx, word in enumerate(vocab)}

    # Afficher quelques mots du vocabulaire pour vérification
    print("Quelques mots du vocabulaire : ", vocab[:10])

    return word_to_index
word_to_index = create_vocab()

Quelques mots du vocabulaire :  ['aw', 'that', 'got', 'david', 'day', 'upset', 'cant', 'updat', 'facebook', 'text']


# Extraction de caractéristiques

In [ ]:

def extract_features_binary_batch(dataframe, word_to_index, batch_size=1000):
    num_rows = len(dataframe)
    num_batches = (num_rows + batch_size - 1) // batch_size

    features = []
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, num_rows)
        batch_df = dataframe.iloc[start_idx:end_idx]
        batch_features = []

        for tokens in batch_df['tokens']:
            word_features = [1 if word in tokens else 0 for word in word_to_index]
            batch_features.append(word_features)

        features.extend(batch_features)

    return features

# Utilisation de la fonction pour extraire les caractéristiques binaires
features_binary = extract_features_binary_batch(dataframe, word_to_index)

# Créer un DataFrame avec les cibles et les caractéristiques binaires
data_binary = pd.DataFrame(features_binary, columns=[f'feat_{i}' for i in range(len(word_to_index))])
data_binary.insert(0, 'target', dataframe['target'])

# Enregistrer le DataFrame dans un fichier CSV
data_binary.to_csv('features_binary.csv', index=False, header=False)


In [10]:
import numpy as np
import pandas as pd

def extract_features_count_batch(dataframe, word_to_index, batch_size=1000):
    num_samples = len(dataframe)
    num_batches = (num_samples + batch_size - 1) // batch_size

    # Parcourir chaque lot (batch) de tweets
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, num_samples)

        # Initialiser un tableau pour stocker les caractéristiques par comptage du lot actuel
        batch_features = np.zeros((end_idx - start_idx, len(word_to_index)), dtype=int)

        # Parcourir chaque tweet du lot
        for i, tokens in enumerate(dataframe['tokens'].iloc[start_idx:end_idx]):
            # Compter le nombre d'apparitions de chaque mot dans le tweet
            for word in tokens:
                if word in word_to_index:
                    word_index = word_to_index[word]
                    batch_features[i, word_index] += 1

        # Créer un DataFrame pour le lot actuel
        batch_df = pd.DataFrame(batch_features, columns=[f'count_{i}' for i in range(len(word_to_index))])
        batch_df.insert(0, 'target', dataframe['target'].iloc[start_idx:end_idx].values)

        # Écrire le lot actuel dans un fichier CSV (mode append)
        if batch_idx == 0:
            batch_df.to_csv('features_count.csv', index=False, header=False)
        else:
            batch_df.to_csv('features_count.csv', index=False, header=False, mode='a')

        # Libérer la mémoire du lot actuel
        del batch_features
        del batch_df

batch_size = 1000

# Utilisation de la fonction pour extraire les caractéristiques par comptage par lots
extract_features_count_batch(dataframe, word_to_index, batch_size=batch_size)


# Classification

Partie 1 : Chargement de données

In [6]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.neighbors import KNeighborsClassifier

In [7]:
#Chargement de données (Résultat de la représentation binaire)
data_features_binary = pd.read_csv('./dataset/features_binary.csv', encoding='latin1', header=None)
data_features_binary.head()

,0,1,2,3,4,5,6,7,8,9,...,842,843,844,845,846,847,848,849,850,851
0,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data_features_binary[0].value_counts()

0
0    800000
4    800000
Name: count, dtype: int64

Convertir le target 4 à 1

In [9]:
data_features_binary[0] = data_features_binary[0].replace(4,1)

In [10]:
data_features_binary[0].value_counts()

0
0    800000
1    800000
Name: count, dtype: int64

In [11]:
# Sélectionner toutes les colonnes sauf la première
X_Binary = data_features_binary.iloc[:, 1:]

# Sélectionner la première colonne
Y_Binary = data_features_binary.iloc[:, 0]

In [12]:
X_Binary.shape, Y_Binary.shape

((1600000, 851), (1600000,))

In [13]:
# Séparation des données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X_Binary, Y_Binary, test_size=0.3, stratify=Y_Binary, random_state=2)

Partie 2 : KNN

In [10]:
# Créer un modèle KNN avec k=5 (vous pouvez ajuster ce paramètre)
knn_model = KNeighborsClassifier(n_neighbors=3)

In [11]:
# Entraîner le modèle sur l'ensemble d'entraînement
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [12]:
# Prédiction sur l'ensemble de test
y_pred = knn_model.predict(X_test)

In [ ]:
# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

In [ ]:
recall = recall_score(y_test, y_pred)
print(recall)

In [ ]:
precision = precision_score(y_test, y_pred)
print(precision)

Partie 3 : Regression Logistique

In [11]:
# Initialisation du modèle de régression logistique avec régularisation L2 (Ridge)
# et un nombre maximal d'itérations de 10000
model = LogisticRegression(penalty="l2", max_iter=1000)

In [12]:
#Entraîne le modèle sur l'ensemble d'entraînement
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [20]:
# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

In [21]:
# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[169321  70679]
 [ 49130 190870]]


In [16]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.7503979166666667


In [17]:
recall = recall_score(y_test, y_pred)
print(recall)

0.7952916666666666


In [18]:
precision = precision_score(y_test, y_pred)
print(precision)

0.7297676534798451
